# Deep Learning with Python - PyTorch Version

This notebook demonstrates the mathematical building blocks of neural networks using **PyTorch** instead of Keras/TensorFlow. It performs the same MNIST classification task as the original Keras version.

**Key Differences from Keras Version:**
- Uses PyTorch tensors and datasets
- Manual training loop implementation
- No high-level Keras abstractions
- Direct gradient computation and optimization

## PyTorch 버전 확인

In [2]:
# PyTorch 버전 확인
try:
    import torch
    import torchvision
    print(f"PyTorch version: {torch.__version__}")
    print(f"TorchVision version: {torchvision.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError:
    print("❌ PyTorch가 설치되지 않았습니다.")

PyTorch version: 2.5.1
TorchVision version: 0.20.1
CUDA available: True


## Load and Explore the MNIST Dataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import numpy as np

In [49]:
# MNIST 데이터셋 로드 (PyTorch 방식)
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5,), (0.5,))  # [0,1] -> [-1,1] 정규화
])

# 훈련 및 테스트 데이터셋 로드
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

# 데이터를 numpy 배열로 변환 (원본 Keras 코드와 동일한 형태로)
train_images = train_dataset.data.numpy()
train_labels = train_dataset.targets.numpy()
test_images = test_dataset.data.numpy()
test_labels = test_dataset.targets.numpy()

print(f"Train images shape: {train_images.shape}")
print(f"Train labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")

Train images shape: (60000, 28, 28)
Train labels shape: (60000,)
Test images shape: (10000, 28, 28)
Test labels shape: (10000,)


In [50]:
print(f"Training labels: {train_labels[:20]}")

Training labels: [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]


In [51]:
print(f"Test labels: {test_labels[:20]}")

Test labels: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


## Build a Simple Neural Network Model

In [54]:
# PyTorch 신경망 모델 정의 (Keras Sequential과 동일한 구조)
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 첫 번째 은닉층: 784 -> 512, ReLU 활성화
        self.fc1 = nn.Linear(28 * 28, 512)
        # 출력층: 512 -> 10, Softmax는 loss 함수에서 처리
        self.fc2 = nn.Linear(512, 10)
        
    def forward(self, x):
        # 입력을 평면화 (28x28 -> 784)
        x = x.view(-1, 28 * 28)
        # 첫 번째 층 + ReLU
        x = F.relu(self.fc1(x))
        # 출력층 (softmax는 loss에서 처리)
        x = self.fc2(x)
        return x

In [55]:
# 모델 인스턴스 생성
model = SimpleNN()
print("모델 구조:")
print(model)

모델 구조:
SimpleNN(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [56]:
# 파라미터 수 계산 (자세한 분석)
print("📊 모델 파라미터 상세 분석:")
print("=" * 50)

total_params = 0
for name, param in model.named_parameters():
    param_count = param.numel()
    total_params += param_count
    print(f"{name:15} | Shape: {str(param.shape):15} | Parameters: {param_count:>7,}")

print("=" * 50)
print(f"총 파라미터 수: {total_params:,}")

# 수동 계산으로 검증
print(f"\n🔍 수동 계산 검증:")
fc1_weights = 28 * 28 * 512  # 784 * 512
fc1_bias = 512
fc2_weights = 512 * 10
fc2_bias = 10

print(f"fc1 가중치: {fc1_weights:,} (784 × 512)")
print(f"fc1 편향:   {fc1_bias:,}")
print(f"fc2 가중치: {fc2_weights:,} (512 × 10)")
print(f"fc2 편향:   {fc2_bias:,}")
print(f"총합:       {fc1_weights + fc1_bias + fc2_weights + fc2_bias:,}")

📊 모델 파라미터 상세 분석:
fc1.weight      | Shape: torch.Size([512, 784]) | Parameters: 401,408
fc1.bias        | Shape: torch.Size([512]) | Parameters:     512
fc2.weight      | Shape: torch.Size([10, 512]) | Parameters:   5,120
fc2.bias        | Shape: torch.Size([10]) | Parameters:      10
총 파라미터 수: 407,050

🔍 수동 계산 검증:
fc1 가중치: 401,408 (784 × 512)
fc1 편향:   512
fc2 가중치: 5,120 (512 × 10)
fc2 편향:   10
총합:       407,050


## Compile the Model (PyTorch Configuration)

In [57]:
# PyTorch에서 모델 구성 (Keras compile과 유사)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실 함수: CrossEntropyLoss (sparse_categorical_crossentropy와 동일)
criterion = nn.CrossEntropyLoss()

# 옵티마이저: Adam
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Device: {device}")
print(f"Loss function: {criterion}")
print(f"Optimizer: {optimizer}")

Device: cuda
Loss function: CrossEntropyLoss()
Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


## Preprocess the Data

In [61]:
# 데이터 전처리 (Keras 버전과 동일)
# 이미지를 평면화하고 정규화
train_images_processed = train_images.reshape((60000, 28 * 28)).astype(np.float32) / 255.0
test_images_processed = test_images.reshape((10000, 28 * 28)).astype(np.float32) / 255.0

# NumPy 배열을 PyTorch 텐서로 변환
train_images_tensor = torch.FloatTensor(train_images_processed)
train_labels_tensor = torch.LongTensor(train_labels)
test_images_tensor = torch.FloatTensor(test_images_processed)
test_labels_tensor = torch.LongTensor(test_labels)

# 데이터셋과 데이터로더 생성
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"전처리된 훈련 데이터 형태: {train_images_processed.shape}")
print(f"전처리된 테스트 데이터 형태: {test_images_processed.shape}")
print(f"배치 크기: 128")
print(f"훈련 배치 수: {len(train_loader)}")
print(f"테스트 배치 수: {len(test_loader)}")

전처리된 훈련 데이터 형태: (60000, 784)
전처리된 테스트 데이터 형태: (10000, 784)
배치 크기: 128
훈련 배치 수: 469
테스트 배치 수: 79


## Train the Neural Network

In [62]:
# 훈련 함수 정의
def train_model(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            # 데이터를 디바이스로 이동
            data, target = data.to(device), target.to(device)
            
            # 그래디언트 초기화
            optimizer.zero_grad()
            
            # 순전파
            output = model(data)
            
            # 손실 계산
            loss = criterion(output, target)
            
            # 역전파
            loss.backward()
            
            # 가중치 업데이트
            optimizer.step()
            
            # 통계 업데이트
            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            
            # 진행상황 출력 (매 100 배치마다)
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch+1}/{epochs}, Batch {batch_idx}/{len(train_loader)}, '
                      f'Loss: {loss.item():.4f}, Accuracy: {100 * correct / total:.2f}%')
        
        # 에포크 종료 시 통계 출력
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f'Epoch {epoch+1} 완료 - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%\n')

# 모델 훈련 실행 (5 에포크)
print("신경망 훈련 시작...")
train_model(model, train_loader, criterion, optimizer, epochs=5)

신경망 훈련 시작...
Epoch 1/5, Batch 0/469, Loss: 2.3080, Accuracy: 11.72%
Epoch 1/5, Batch 0/469, Loss: 2.3080, Accuracy: 11.72%
Epoch 1/5, Batch 100/469, Loss: 0.3546, Accuracy: 83.61%
Epoch 1/5, Batch 100/469, Loss: 0.3546, Accuracy: 83.61%
Epoch 1/5, Batch 200/469, Loss: 0.1627, Accuracy: 87.55%
Epoch 1/5, Batch 200/469, Loss: 0.1627, Accuracy: 87.55%
Epoch 1/5, Batch 300/469, Loss: 0.1602, Accuracy: 89.53%
Epoch 1/5, Batch 300/469, Loss: 0.1602, Accuracy: 89.53%
Epoch 1/5, Batch 400/469, Loss: 0.2466, Accuracy: 90.71%
Epoch 1 완료 - Loss: 0.3193, Accuracy: 91.29%

Epoch 2/5, Batch 0/469, Loss: 0.1815, Accuracy: 95.31%
Epoch 1/5, Batch 400/469, Loss: 0.2466, Accuracy: 90.71%
Epoch 1 완료 - Loss: 0.3193, Accuracy: 91.29%

Epoch 2/5, Batch 0/469, Loss: 0.1815, Accuracy: 95.31%
Epoch 2/5, Batch 100/469, Loss: 0.2646, Accuracy: 95.61%
Epoch 2/5, Batch 100/469, Loss: 0.2646, Accuracy: 95.61%
Epoch 2/5, Batch 200/469, Loss: 0.1179, Accuracy: 95.92%
Epoch 2/5, Batch 200/469, Loss: 0.1179, Accuracy: 

## Make Predictions on Test Data

In [63]:
# 테스트 데이터에서 예측 수행 (Keras 버전과 동일한 동작)
model.eval()  # 평가 모드로 설정

with torch.no_grad():
    # 첫 10개 테스트 이미지에 대한 예측
    test_digits = test_images_tensor[0:10].to(device)
    predictions = model(test_digits)
    
    # Softmax 적용하여 확률로 변환
    predictions_prob = F.softmax(predictions, dim=1)
    
    print("첫 번째 테스트 이미지의 예측 확률:")
    print(predictions_prob[0].cpu().numpy())
    print(f"\n예측 확률의 합: {predictions_prob[0].sum().item():.4f}")

첫 번째 테스트 이미지의 예측 확률:
[8.6196195e-08 1.4769905e-08 3.4010934e-06 8.9753339e-05 4.1802733e-10
 2.1066103e-07 2.4664220e-12 9.9989688e-01 3.3224021e-06 6.2825375e-06]

예측 확률의 합: 1.0000


## Analyze Prediction Results

In [64]:
# 예측 결과 분석 (Keras 버전과 동일)
with torch.no_grad():
    # 가장 높은 확률의 클래스 찾기 (argmax)
    predicted_class = torch.argmax(predictions_prob[0]).item()
    print(f"예측된 클래스: {predicted_class}")
    
    # 7번 클래스의 확률
    prob_class_7 = predictions_prob[0][7].item()
    print(f"클래스 7의 확률: {prob_class_7:.6f}")
    
    # 실제 레이블
    actual_label = test_labels[0]
    print(f"실제 레이블: {actual_label}")
    
    # 예측 정확도 확인
    is_correct = predicted_class == actual_label
    print(f"예측 정확성: {'✅ 맞음' if is_correct else '❌ 틀림'}")

예측된 클래스: 7
클래스 7의 확률: 0.999897
실제 레이블: 7
예측 정확성: ✅ 맞음


## Evaluate Model Performance

In [65]:
# 전체 테스트 세트에 대한 모델 성능 평가
def evaluate_model(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            # 손실 누적
            test_loss += criterion(output, target).item()
            
            # 정확도 계산
            _, predicted = torch.max(output, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    # 평균 손실과 정확도 계산
    avg_loss = test_loss / len(test_loader)
    accuracy = correct / total
    
    return avg_loss, accuracy

# 모델 평가 실행
test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)

print(f"테스트 결과:")
print(f"테스트 손실 (test_loss): {test_loss:.4f}")
print(f"테스트 정확도 (test_acc): {test_acc:.4f}")
print(f"테스트 정확도 (백분율): {test_acc * 100:.2f}%")

테스트 결과:
테스트 손실 (test_loss): 0.0652
테스트 정확도 (test_acc): 0.9798
테스트 정확도 (백분율): 97.98%


## 📊 결과 비교

### PyTorch vs Keras 구현 비교:

| 항목 | Keras | PyTorch |
|------|-------|----------|
| **모델 정의** | `Sequential([layers.Dense(...)])` | `class SimpleNN(nn.Module)` |
| **모델 컴파일** | `model.compile(optimizer, loss, metrics)` | 별도로 `criterion`, `optimizer` 정의 |
| **훈련** | `model.fit(X, y, epochs, batch_size)` | 수동 훈련 루프 구현 |
| **예측** | `model.predict(X)` | `model(X)` + `F.softmax()` |
| **평가** | `model.evaluate(X, y)` | 수동 평가 함수 구현 |

### 장단점:

**Keras 장점:**
- 간단하고 직관적인 API
- 적은 코드로 빠른 프로토타이핑
- 자동화된 훈련/평가 과정

**PyTorch 장점:**
- 더 세밀한 제어 가능
- 훈련 과정의 모든 단계를 명시적으로 구현
- 연구 및 실험에 더 적합
- 동적 그래프 지원

두 구현 모두 동일한 결과를 산출해야 하며, 약 97-98%의 테스트 정확도를 달성할 것으로 예상됩니다.